In [1]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import datetime
from keras.models import Sequential, load_model
from tensorflow.keras import layers, regularizers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from scipy import misc, ndimage
from PIL import Image
from keras.optimizers import Adam, RMSprop
import time
from IPython.display import clear_output
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})
from keras.regularizers import l2, l1, l1_l2
from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)
from sklearn.metrics import classification_report, confusion_matrix
import requests
import json
import glob
from keras.callbacks import ModelCheckpoint, EarlyStopping, RemoteMonitor, LambdaCallback
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mimg
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,7)
import pandas as pd
from scipy import misc
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import random
from sklearn.model_selection import KFold

In [2]:
def tg_sendtext(bot_message):

    bot_token = ''
    bot_chatID = '1220107026'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

def message_discord(message):
    channelID = "777064829063135234" # enable dev mode on discord, right-click on the channel, copy ID
    botToken = "Nzc3MDYzODcyMjgzNTQxNTE0.X69_IA.TtsQh9eVhxFfeakGkQLqn9UDAxw"    # get from the bot page. must be a bot, not a discord app

    baseURL = "https://discordapp.com/api/channels/{}/messages".format(channelID)
    headers = { "Authorization":"Bot {}".format(botToken),
                "User-Agent":"myBotThing (http://some.url, v0.1)",
                "Content-Type":"application/json", }

    POSTedJSON =  json.dumps ( {"content":message} )

    r = requests.post(baseURL, headers = headers, data = POSTedJSON)

def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [3]:
def check_accuracy(mdl, X_val, y_val):
    true = 0
    Y_pred = mdl.predict(X_val)
    y_pred = np.argmax(Y_pred, axis=1)
    for j in range(len(y_val)):
        if y_pred[j] == y_val[j]:
            true += 1
    accc = (true/len(y_new))*100
    return accc

In [4]:
def check_highest(folder_path1, train_max, max_acc, max_i):
    temp_max_acc = [0]
    temp_max_i = [0]
    for i in os.listdir(folder_path1):
        model = load_model(folder_path1 + i, custom_objects = {"Activation": Activation})
        acc = check_accuracy(model, X_new, y_new)
        print('Model Accuracy =', str(acc) + '%')
        if acc > 92:
            os.remove(folder_path1 + i)
            new_name = i.replace(".h5","") + '_' + str(round(acc, 2)) + '.h5'
            model.save('good_models/' + new_name)
            temp_max_acc.append(acc)
            temp_max_i.append(new_name)
        else:
            os.remove(folder_path1 + i)
    max_acc = np.append(max_acc, temp_max_acc)
    max_i = np.append(max_i, temp_max_i)
    maxx = np.amax(max_acc)
    index = np.where(max_acc == maxx)
    if maxx > train_max:
        train_max = maxx
        print('New High Accuracy =', train_max)
        mess = str(train_max)
        tg_sendtext('New High Accuracy = ' + mess)
        message_discord('New High Accuracy = ' + mess)
    print('The highest Accuracy:', max_i[index], 'with', str(maxx) + '%')
    return train_max, acc, max_acc, max_i

In [5]:
def check_highest_custom(folder_path1):
    for i in os.listdir(folder_path1):
        model = load_model(folder_path1 + i, custom_objects = {"Activation": Activation})
        acc = check_accuracy(model, X_new, y_new)
        if acc > 92:
            os.remove(folder_path1 + i)
            new_name = str(round((random.uniform(0, 10) + random.uniform(0, 10)) * random.uniform(0, 100), 5)) + i.replace(".h5","") + '.h5'
            model.save('checkpoints/' + new_name)
        else:
            os.remove(folder_path1 + i)

In [6]:
# from google.colab import drive, output

# drive.mount('/content/gdrive')
root_path = 'pothole_detection'

dataset = 'dataset'
categories = ['pothole', 'no_pothole']
test_categories = ['test_pothole', 'test_no_pothole']
image_size = 120
data = []
data_test = []

In [7]:
cnt = 0
for category in categories:
    folder = os.path.join(dataset, category)
    label = categories.index(category)
    for img in listdir_nohidden(folder):
        try:
            img_path = os.path.join(folder, img)
            cnt+=1
            print(category, cnt, label)
            clear_output(wait = True)
            img_array = cv2.imread(img_path)
            img_array = cv2.resize(img_array, (image_size, image_size))
            data.append([img_array, label])
        except Exception as e:
            print(str(e))

for category in test_categories:
    folder = os.path.join(dataset, category)
    label = test_categories.index(category)
    for img in listdir_nohidden(folder):
        try:
            img_path = os.path.join(folder, img)
            cnt+=1
            print(category, cnt, label)
            clear_output(wait = True)
            img_array = cv2.imread(img_path)
            img_array = cv2.resize(img_array, (image_size, image_size))
            data_test.append([img_array, label])
        except Exception as e:
            print(str(e))          

test_no_pothole 15205 1


In [8]:
np.random.shuffle(data)
np.random.shuffle(data_test)

X_old = []
y_old = []
X_old_test = []
y_old_test = []

for features, labels in data:
    X_old.append(features)
    y_old.append(labels)

X_train = np.array(X_old)
y_train = np.array(y_old)
X_train = X_train/225

for features, labels in data_test:
    X_old_test.append(features)
    y_old_test.append(labels)

X_test = np.array(X_old_test)
y_test = np.array(y_old_test)
X_test = X_test/255

In [9]:
root_path = ''
dataset = 'dataset'
categories = ['pothole', 'no_pothole']
image_size = 120

folder_path = 'validation'
data_new = []

cnt = 0
for category in categories:
    folder_new = os.path.join(dataset, folder_path, category)
    label_new = categories.index(category)
    for img_new in os.listdir(folder_new):
        img_path_new = os.path.join(folder_new, img_new)
        cnt+=1
        print(category, cnt, label_new)
        clear_output(wait = True)
        img_array_new = cv2.imread(img_path_new)
        img_array_new = cv2.resize(img_array_new, (image_size, image_size))
        data_new.append([img_array_new, label_new])

np.random.shuffle(data_new)
X_old = []
y_old = []

for features_new, labels_new in data_new:
    X_old.append(features_new)
    y_old.append(labels_new)

X_new = np.array(X_old)
y_new = np.array(y_old)
X_new = X_new/225

no_pothole 614 1


# **Cross Validation**

In [10]:
num_folds = 10
acc_per_fold = []
loss_per_fold = []
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)
kfold = KFold(n_splits=num_folds, shuffle=True)

In [11]:
def train_cross_val(input_layer_neurons_val, first_layer_neurons_val, second_layer_neurons_val, mes):
    model = Sequential()
    fold_no = 1
    pat = 1
    early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)
    model_checkpoint_best = ModelCheckpoint('saved_models/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=0, save_best_only=True, monitor='val_loss')
    
    model_checkpoint = ModelCheckpoint('temp_checkpoint/' + "model_{epoch:02d}" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=0, save_best_only=False, monitor='val_loss')
    
    log_dir = "logs/fit/" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S")
    logs = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    model.add(Conv2D(input_layer_neurons_val, (3, 3), kernel_initializer='random_normal', kernel_regularizer = l1(0.0005), activity_regularizer = l1(0.0005), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    model.add(Dense(first_layer_neurons_val, input_shape = X_train.shape[1:], kernel_initializer='random_normal', activation = 'relu'))
    model.add(Dropout(0.05))

    model.add(Dense(second_layer_neurons_val, input_shape = X_train.shape[1:], kernel_initializer='random_normal', activation = 'relu'))
    model.add(Dropout(0.05))

    model.add(Dense(2, activation = 'softmax'))

    ###

    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    ###
    
    print('training')
    print('Fold Num =', end=' ')
    for train, test in kfold.split(inputs, targets):
        print(fold_no)  
        
        model.fit(inputs[train], targets[train], epochs = 5, validation_split = 0.1, batch_size = 32, shuffle=True, verbose = 1, callbacks = [logs, early_stopping, model_checkpoint, model_checkpoint_best])

        scores = model.evaluate(inputs[test], targets[test], verbose=0)
        
        tg_sendtext(f'Finished fold number {fold_no} of {mes} with accuracy = {scores[1]*100}%')
        message_discord(f'Finished fold number {fold_no} of {mes} with accuracy = {scores[1]*100}%')
        
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
        fold_no = fold_no + 1
        model.save('saved_models/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5')
        check_highest_custom('temp_checkpoint/')
    print('Finished')

# **Auto Trainer**

In [12]:
input_layer_neurons = list(np.arange(18, 25))
first_layer_neurons = list(np.arange(12, 17))
second_layer_neurons = list(np.arange(12, 17))
res = [[i, j, k]
       for i in input_layer_neurons
       for j in first_layer_neurons
       for k in second_layer_neurons]
res = res[::-1]
res.remove([24, 16, 16])
res.remove([24, 16, 15])

maxx = 0
count = 3
maxx_acc = []
maxx_i = []
for i in res:
    print(count)
    neurons = [i[0], i[1], i[2]]
    print('Neurons =', i[0], i[1], i[2])
    message = 'model ' + str(count)
    tg_sendtext('Started training ' + message + ' with neurons = ' + str(neurons))
    message_discord('Started training ' + message + ' with neurons = ' + str(neurons))
    train_cross_val(i[0], i[1], i[2], message)
    time.sleep(5)
    maxx, accuracy, maxx_acc, maxx_i = check_highest('saved_models/', maxx, maxx_acc, maxx_i)
    message = message + ' ' + str([i[0], i[1], i[2]]) + ' with accuracy = ' + str(accuracy)
    tg_sendtext('Finished training ' + message)
    message_discord('Finished training ' + message)
    count += 1
    print('------------------------------------------------------------------------')
    tg_sendtext('------------------------------------------------------------------------')
    message_discord('------------------------------------------------------------------------')
print('AHHHHHHH')

3
Neurons = 24 16 14
training
Fold Num = 1
Epoch 1/5
  1/385 [..............................] - ETA: 0s - loss: 10.5777 - accuracy: 0.4375WARNING:tensorflow:From /Users/partthshah/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
385/385 [==============================] - 60s 155ms/step - loss: 0.7211 - accuracy: 0.7343 - val_loss: 0.5919 - val_accuracy: 0.7619
Epoch 2/5
385/385 [==============================] - 54s 140ms/step - loss: 0.4257 - accuracy: 0.8296 - val_loss: 0.4986 - val_accuracy: 0.7772
Epoch 3/5
385/385 [==============================] - 56s 146ms/step - loss: 0.3274 - accuracy: 0.8753 - val_loss: 0.4595 - val_accuracy: 0.8247
Epoch 4/5
385/385 [==============================] - 53s 138ms/step - loss: 0.2341 - accuracy: 0.9206 - val_loss: 0.5525 - val_accuracy: 0.82

KeyboardInterrupt: 

In [ ]:
message = 'Model 1'
train_cross_val(21, 14, 14, message)

In [ ]:
n_folds = 10
mes = 'Model 2'
pat = 3
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)
model_checkpoint_best = ModelCheckpoint('checkpoints/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=1, save_best_only=True)
model_checkpoint = ModelCheckpoint('temp_checkpoint/' + "model_{epoch:02d}" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5', verbose=0, save_best_only=False, monitor='val_loss')
model_history = []
log_dir = "logs/fit/" + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S")
logs = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
### Model ###
model = Sequential()

model.add(Conv2D(128, (3, 3), kernel_initializer='random_normal', kernel_regularizer = l2(0.0009), activity_regularizer = l2(0.0005), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(1024, input_shape = X_train.shape[1:], kernel_initializer='random_normal', kernel_regularizer = l2(0.0009), activity_regularizer = None, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(512, input_shape = X_train.shape[1:], kernel_initializer='random_normal', kernel_regularizer = l2(0.0009), activity_regularizer = None, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(256, input_shape = X_train.shape[1:], kernel_initializer='random_normal', kernel_regularizer = l2(0.0009), activity_regularizer = None, activation = 'relu'))
model.add(Dropout(0.3))

model.add(Dense(128, input_shape = X_train.shape[1:], kernel_initializer='random_normal', kernel_regularizer = l2(0.0009), activity_regularizer = None, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(64, input_shape = X_train.shape[1:], kernel_initializer='random_normal', kernel_regularizer = l2(0.0009), activity_regularizer = None, activation = 'relu'))
model.add(Dropout(0.1))

model.add(Dense(32, input_shape = X_train.shape[1:], kernel_initializer='random_normal', kernel_regularizer = l2(0.0009), activity_regularizer = None, activation = 'relu'))
model.add(Dropout(0.08))

model.add(Dense(16, input_shape = X_train.shape[1:], kernel_initializer='random_normal', kernel_regularizer = l2(0.0009), activity_regularizer = None, activation = 'relu'))
model.add(Dropout(0.05))

model.add(Dense(2, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

print('training')
print('Fold Num =', end=' ')
for i in range(n_folds):
    print(i+1, end=' ')
    t_x, val_x, t_y, val_y = train_test_split(inputs, targets, test_size=0.2,
                                               random_state = np.random.randint(1,1000, 1)[0])

    model.fit(t_x, t_y, epochs=10, batch_size=64, callbacks=[early_stopping, model_checkpoint, model_checkpoint_best, logs], verbose=1, validation_split=0.2)

    scores = model.evaluate(val_x, val_y, verbose=0)

    tg_sendtext(f'Finished fold number {i+1} of {mes} with accuracy = {scores[1]*100}%')
    message_discord(f'Finished fold number {i+1} of {mes} with accuracy = {scores[1]*100}%')
    model.save('saved_models/' + (datetime.datetime.utcnow()+datetime.timedelta(hours=5.5)).strftime("%d-%m-%Y--%H.%M.%S") + '.h5')
    check_highest_custom('temp_checkpoint/')
print('Finished')

training
Fold Num = 1 Epoch 1/10
  4/153 [..............................] - ETA: 3:55:31 - loss: 990.3535 - accuracy: 0.5273 